# Importing Libraries

In [48]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text

# Reading data

In [49]:
df_input = pd.read_csv('TestInput.csv')
df_input

,SUPPLIER_ERP,SUPPLIER_ADDRESS,SUPPLIER_CITY,SUPPLIER_STATE,SUPPLIER_ZIP_POSTAL_CDE,SUPPLIER_COUNTRY
0,BRAINBOX DESIGN ESTRATEGICO S A,NaN,CURITBA,NaN,NaN,BRASIL
1,CHRISTIAN D STORZ,FRANCHISE PERFORMANCE 9 CORONET TERRACE,BURLINGTON,NJ,8016,US
2,COOPERATIVE FOR ASSISTANCE AND RELIEF EVERYWHE...,151 ELLIS STREET NE,ATLANTA,GA,30303,US
3,FWH,325 SUB WAY,MILFORD,CT,6461,USA
4,INTERNATIONAL ASSOCIATION OF IT ASSET MANAGERS,IAITAM 4848 MUNSON STREET NW,CANTON,OH,44718,US
5,MATTHEW MILLER,BUSINESS ANALYTICS 266 LEMUR DRIVE,SAN ANTONIO,TX,78213,US
6,NEXO,NaN,BRUSSELS,NaN,NaN,BELGUIM
7,"REINHART FOODSERVICE, L.L.C.-WISCONSIN",100 HARBOR VIEW PLAZA SUITE 200,LA CROSSE,WI,54601,US
8,WIGGIN AND DANA LLP,ATTENTION: ACCOUNTS RECEIVABLE P.O. BOX 1832,NEW HAVEN,CT,06508-1832,US
9,"WILLIAM MORRIS ENDEAVOR ENTERTAINMENT, LLC",1201 DEMONBREUN STREET 15TH FLOOR,NASHVILLE,TN,37203,US


In [50]:
df_master = pd.read_csv('TestMaster.csv')
df_master

,D-U-N-S Number,Business Name,Location Type,Web Address,Telephone Number,Year Started,Currency Code,Status Code,Population Code,Last Update Date,...,Country/Region Name,Global Ultimate D-U-N-S Number,Global Ultimate Business Name,Parent D-U-N-S Number,Major Industry Category,Major Industry Category Name,Related Industries,Line of Business,Client,File Name
0,927477331,"William Morris Endeavor Entertainment, LLC",Branch,http://www.wmeagency.com,6.159633e+09,0,NaN,2,9,19971003,...,USA,117081770,"Endeavor Group Holdings, Inc.",NaN,9.0,Services,Theaters,Theatrical producers/services,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
1,137673179,International Association of Information Techn...,HQ,http://www.iaitam.org,3.306283e+09,2002,20.0,1,8,20160404,...,USA,137673179,International Association of Information Techn...,NaN,9.0,Services,Consumer Electronics & Computers;Education,Computer related services,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
2,120835500,Petras LLC,Single Location,NaN,NaN,2022,20.0,0,7,20221024,...,USA,120835500,Petras LLC,NaN,NaN,Public Administration,Other,Nonclassified establishment,AAR,AAR Supplier to D&amp;amp;B 2023 ytd_full_2023...
3,113036002,T & S Design LLC,Single Location,NaN,NaN,2019,20.0,0,9,20221112,...,USA,113036002,T & S Design LLC,NaN,9.0,Services,Business Services,Business services,GAP,GAP Suppliers-DnB_Upload_full_2023-07-18_16-47...
4,104955367,Vivian Storz,Single Location,NaN,6.097843e+09,2017,20.0,0,8,20210626,...,USA,104955367,Vivian Storz,NaN,8.0,"Finance, Insurance, Real Estate",Real Estate,Nonresidential building operator,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
5,82672270,Wiggin & Dana LLP,HQ,http://www.wiggin.com,2.034984e+09,1934,20.0,1,9,20210628,...,USA,82672270,Wiggin & Dana LLP,NaN,9.0,Services,Law Firms & Legal Services,Legal services office,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
6,47191968,"Reinhart Foodservice, L.L.C.",Branch,http://www.performancefoodservice.com,6.087939e+09,0,NaN,2,7,20200421,...,USA,79525714,Performance Food Group Company,NaN,6.0,Wholesale Trade,Grocery,Whol general groceries,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
7,36768041,Methodist Plaza Imaging Ctr,Single Location,NaN,NaN,1991,20.0,0,9,20120727,...,USA,36768041,Methodist Plaza Imaging Ctr,NaN,9.0,Services,Non-Profit & Professional Orgs.,Religious organization,AAR,AAR Supplier to D&amp;amp;B July (Lawson) and ...
8,34213823,"336 Bic Drive Realty II, LLC",Single Location,NaN,NaN,2009,20.0,0,9,20230204,...,USA,34213823,"336 Bic Drive Realty II, LLC",NaN,8.0,"Finance, Insurance, Real Estate",Real Estate,Real estate agent/manager,SUB,SUB SUB_DNB_SUPPLIER_LIST_full_2023-09-28_19-0...
9,1793082,Cooperative For Assistance and Relief Everywhe...,HQ,http://www.care.org,4.046813e+09,1946,20.0,1,9,20221107,...,USA,1793082,Cooperative For Assistance and Relief Everywhe...,NaN,9.0,Services,Non-Profit & Professional Orgs.,Civic/social association,GAP,GAP gap_ap_supnormreboot_D&B_full_2023-07-18_1...


# Clean data

In [51]:
df_input.loc[(df_input['SUPPLIER_ADDRESS']==',,') | (df_input['SUPPLIER_ADDRESS']==',,,'),['SUPPLIER_ADDRESS']] = ''

In [52]:
df_input['SUPPLIER_ADDRESS'] = df_input['SUPPLIER_ADDRESS'] + " " + df_input['SUPPLIER_CITY'] + " " +  df_input['SUPPLIER_STATE'] 

In [53]:
df_input['SUPPLIER_ERP'] = df_input['SUPPLIER_ERP'].fillna('UnknownSupplierERP')
df_input['SUPPLIER_ADDRESS'] = df_input['SUPPLIER_ADDRESS'].fillna('UnknownSupplierAddress')

In [54]:
df_master['Street Address'] = df_master['Street Address'].fillna('') + " " + df_master['City Name'].fillna('') + " " +  df_master['State Province Abbreviation'].fillna('') 

# Additional functions and datasets to process data

In [55]:
def clean_names(document):
    
    document = document.lower()
    
    document = document.replace("."," ")
    document = document.replace("*"," ")
    document = document.replace("-"," ")
    document = document.replace(","," ")
    document = document.replace("["," ")
    document = document.replace("]"," ")
    document = document.replace("("," ")
    document = document.replace(")"," ")
    document = " ".join(document.split())
    
    return document

In [56]:
def clean_addresses(document):
    
    document = document.lower()
    
    # Setting abbreviations in addresses
    document = document.replace("avenue", "ave")
    document = document.replace("boulevard","blvd")
    document = document.replace("court","ct")
    document = document.replace("drive","dr")
    document = document.replace("lane","ln")
    document = document.replace("place","pl")
    document = document.replace("road","rd")
    document = document.replace("street","st")
    document = document.replace("square","sq")
    document = document.replace("trail","trl")
    document = document.replace("terrace","ter")
    document = document.replace("parkway","pkwy")
    document = document.replace("highway","hwy")
    document = document.replace("route","rte")
    document = document.replace("apartment","apt")
    document = document.replace("suite","ste")
    document = document.replace("building","bldg")
    document = document.replace("floor","fl")
    document = document.replace("junction","jct")
    document = document.replace("plaza","plz")
    document = document.replace("room","rm")
    document = document.replace("unit","unit")
    document = document.replace("manor","mnr")
    document = document.replace("center","ctr")
    document = document.replace("crescent","cres")
    document = document.replace("expressway","expwy")
    document = document.replace("crossing","xing")
    document = document.replace("turnpike","tpk")
    
    document = document.replace("."," ")
    document = document.replace("*"," ")
    document = document.replace("-"," ")
    document = document.replace(","," ")
    document = document.replace("["," ")
    document = document.replace("]"," ")
    document = document.replace("("," ")
    document = document.replace(")"," ")
    document = " ".join(document.split())
    
    return document

In [57]:
stop_words_names = list(text.ENGLISH_STOP_WORDS.union(
    ['inc','llp','lp','llc', 'www', 'web','com','net',
     #'solutions','company', 'technologies'
     #,'tech','technology','security','private','limited','association','services','communications','financial','marketing'
     #,'software','group','corp','corporation','corporate','consulting','partners','systems','usa','logistics',
     #'insurance','health','hospital','medical','management','capital','global','coaching','md','international'
     #,'data','america','american','department','dept','direct','associates','business','incorporated', 'hotel', 'hotels',
     #'refreshment','cafe','enterprise','enterprises', 'resource', 'resources','advisors','assoc'
     #,'secure','security','service','foundation','bank','state','hold','holding','holdings','industry','industries'
     #,'analytics','office','project','investments'
    ]
))

# String Matching Algorithm

In [58]:
# Use TF-IDF Vectorizer to create the word vectors as sparse matrix
vectorizer_names =TfidfVectorizer(
            analyzer="word",
            preprocessor = clean_names, 
            #stop_words=stop_words_names,
            token_pattern = r"(?u)\b\S+\b")

business_name_sparse = vectorizer_names.fit_transform(pd.concat([df_master["Business Name"],df_input["SUPPLIER_ERP"]]))
supplier_erp_sparse = vectorizer_names.transform(df_input["SUPPLIER_ERP"])
cosine_supplier_name = cosine_similarity(supplier_erp_sparse,business_name_sparse[0:len(df_master["Business Name"])])

vectorizer_addresses =TfidfVectorizer(
            analyzer="word",
            preprocessor = clean_addresses, 
            stop_words='english',
            token_pattern = r"(?u)\b\S+\b")

street_address_sparse = vectorizer_addresses.fit_transform(pd.concat([df_master["Street Address"],df_input["SUPPLIER_ADDRESS"]]))
supplier_address_sparse = vectorizer_addresses.transform(df_input["SUPPLIER_ADDRESS"])
cosine_supplier_address = cosine_similarity(supplier_address_sparse,street_address_sparse[0:len(df_master["Street Address"])])

# Matrices to check String Matching

In [59]:
#pd.DataFrame(supplier_erp_sparse.todense(), columns=vectorizer_names.get_feature_names_out(), index=df_input["SUPPLIER_ERP"])

In [60]:
pd.DataFrame(cosine_supplier_name, columns=df_master["Business Name"], index=df_input["SUPPLIER_ERP"])

Business Name,"William Morris Endeavor Entertainment, LLC","International Association of Information Technology Asset Managers, Inc.",Petras LLC,T & S Design LLC,Vivian Storz,Wiggin & Dana LLP,"Reinhart Foodservice, L.L.C.",Methodist Plaza Imaging Ctr,"336 Bic Drive Realty II, LLC","Cooperative For Assistance and Relief Everywhere, Inc. (care)"
SUPPLIER_ERP,,,,,,,,,,
BRAINBOX DESIGN ESTRATEGICO S A,0.0,0.000000,0.000000,0.418681,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
CHRISTIAN D STORZ,0.0,0.000000,0.000000,0.000000,0.348522,0.000000,0.000000,0.0,0.000000,0.000000
COOPERATIVE FOR ASSISTANCE AND RELIEF EVERYWHERE INC.,0.0,0.109102,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.914682
FWH,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
INTERNATIONAL ASSOCIATION OF IT ASSET MANAGERS,0.0,0.687529,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
MATTHEW MILLER,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
NEXO,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
"REINHART FOODSERVICE, L.L.C.-WISCONSIN",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.918674,0.0,0.000000,0.000000
WIGGIN AND DANA LLP,0.0,0.000000,0.000000,0.000000,0.000000,0.886865,0.000000,0.0,0.000000,0.148118


In [61]:
#pd.DataFrame(supplier_address_sparse.todense(), columns=vectorizer_addresses.get_feature_names_out(), index=df_input["SUPPLIER_ADDRESS"])

In [62]:
pd.DataFrame(cosine_supplier_address, columns=df_master["Street Address"], index=df_input["SUPPLIER_ADDRESS"])

Street Address,1201 Demonbreun St FL 15 Nashville TN,4848 Munson St NW Canton OH,615 S Pleasant View Dr Mustang OK,1955 Greymont St Philadelphia PA,9 Coronet Ter Burlington NJ,265 Church St FL 14 New Haven CT,100 Harborview Plz La Crosse WI,San Antonio TX,325 Sub Way Milford CT,151 Ellis St Ne Atlanta GA
SUPPLIER_ADDRESS,,,,,,,,,,
UnknownSupplierAddress,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FRANCHISE PERFORMANCE 9 CORONET TERRACE BURLINGTON NJ,0.000000,0.000000,0.000000,0.000000,0.811725,0.000000,0.000000,0.000000,0.000000,0.000000
151 ELLIS STREET NE ATLANTA GA,0.066405,0.072910,0.000000,0.071743,0.000000,0.060730,0.000000,0.000000,0.000000,1.000000
325 SUB WAY MILFORD CT,0.000000,0.000000,0.000000,0.000000,0.000000,0.113243,0.000000,0.000000,1.000000,0.000000
IAITAM 4848 MUNSON STREET NW CANTON OH,0.059634,0.898037,0.000000,0.064428,0.000000,0.054538,0.000000,0.000000,0.000000,0.065476
BUSINESS ANALYTICS 266 LEMUR DRIVE SAN ANTONIO TX,0.000000,0.000000,0.113418,0.000000,0.000000,0.000000,0.000000,0.571759,0.000000,0.000000
UnknownSupplierAddress,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100 HARBOR VIEW PLAZA SUITE 200 LA CROSSE WI,0.000000,0.000000,0.109293,0.000000,0.000000,0.000000,0.633961,0.000000,0.000000,0.000000
ATTENTION: ACCOUNTS RECEIVABLE P.O. BOX 1832 NEW HAVEN CT,0.000000,0.000000,0.000000,0.000000,0.000000,0.280849,0.000000,0.000000,0.092136,0.000000


# Weighted Average

In [63]:
cosine_weighted_average = np.average([cosine_supplier_name, cosine_supplier_address]
                                     , weights=[1,2]
                                     , axis=0)

## Finding best match

In [64]:
cosine_max_combined = np.amax(cosine_weighted_average, axis=1)
cosine_max_combined_index = np.argmax(cosine_weighted_average, axis=1)

# Extract matches

In [65]:
matches = pd.DataFrame({
    'D-U-N-S Number':df_master["D-U-N-S Number"].iloc[cosine_max_combined_index]
    ,"business_name": df_master["Business Name"].iloc[cosine_max_combined_index]
    ,"street_address":df_master["Street Address"].iloc[cosine_max_combined_index]
    ,"cosine":np.round(cosine_max_combined,4)})
matches

,D-U-N-S Number,business_name,street_address,cosine
3,113036002,T & S Design LLC,1955 Greymont St Philadelphia PA,0.1396
4,104955367,Vivian Storz,9 Coronet Ter Burlington NJ,0.6573
9,1793082,Cooperative For Assistance and Relief Everywhe...,151 Ellis St Ne Atlanta GA,0.9716
8,34213823,"336 Bic Drive Realty II, LLC",325 Sub Way Milford CT,0.6667
1,137673179,International Association of Information Techn...,4848 Munson St NW Canton OH,0.8279
7,36768041,Methodist Plaza Imaging Ctr,San Antonio TX,0.3812
0,927477331,"William Morris Endeavor Entertainment, LLC",1201 Demonbreun St FL 15 Nashville TN,0.0000
6,47191968,"Reinhart Foodservice, L.L.C.",100 Harborview Plz La Crosse WI,0.7289
5,82672270,Wiggin & Dana LLP,265 Church St FL 14 New Haven CT,0.4829
0,927477331,"William Morris Endeavor Entertainment, LLC",1201 Demonbreun St FL 15 Nashville TN,0.8673


# Get exact names matches from D&B master

In [66]:
df_supplier_name=pd.DataFrame({'D-U-N-S Number':df_master["D-U-N-S Number"].iloc[np.argmax(cosine_supplier_name, axis=1)]
                                ,'business_name':df_master["Business Name"].iloc[np.argmax(cosine_supplier_name, axis=1)]
                                ,'street_address':df_master["Street Address"].iloc[np.argmax(cosine_supplier_name, axis=1)]
                                ,'cosine':np.max(cosine_supplier_name, axis=1)})
df_supplier_name.reset_index(drop=True, inplace=True)
df_exact_name = df_supplier_name[df_supplier_name['cosine'] >= 1]
df_exact_name

,D-U-N-S Number,business_name,street_address,cosine
9,927477331,"William Morris Endeavor Entertainment, LLC",1201 Demonbreun St FL 15 Nashville TN,1.0


# Replace using exact name matches 

In [67]:
matches.iloc[df_exact_name.index] = df_exact_name

# Output File

In [68]:
result = pd.DataFrame({
    "SUPPLIER_ERP":df_input["SUPPLIER_ERP"].values
    ,"SUPPLIER_ADDRESS":df_input["SUPPLIER_ADDRESS"].values
    ,"SUPPLIER_CITY":df_input["SUPPLIER_CITY"].values
    ,"SUPPLIER_STATE":df_input["SUPPLIER_STATE"].values
    ,"SUPPLIER_ZIP_POSTAL_CDE":df_input["SUPPLIER_ZIP_POSTAL_CDE"].values
    ,"SUPPLIER_COUNTRY":df_input["SUPPLIER_COUNTRY"].values
    ,'D-U-N-S Number':matches["D-U-N-S Number"]
    ,"business_name":matches['business_name']
    ,"street_address":matches['street_address']
    ,"cosine_similarity":matches["cosine"]})

In [69]:
result.sort_values(by='cosine_similarity', ascending=False)

,SUPPLIER_ERP,SUPPLIER_ADDRESS,SUPPLIER_CITY,SUPPLIER_STATE,SUPPLIER_ZIP_POSTAL_CDE,SUPPLIER_COUNTRY,D-U-N-S Number,business_name,street_address,cosine_similarity
0,"WILLIAM MORRIS ENDEAVOR ENTERTAINMENT, LLC",1201 DEMONBREUN STREET 15TH FLOOR NASHVILLE TN,NASHVILLE,TN,37203,US,927477331,"William Morris Endeavor Entertainment, LLC",1201 Demonbreun St FL 15 Nashville TN,1.0000
9,COOPERATIVE FOR ASSISTANCE AND RELIEF EVERYWHE...,151 ELLIS STREET NE ATLANTA GA,ATLANTA,GA,30303,US,1793082,Cooperative For Assistance and Relief Everywhe...,151 Ellis St Ne Atlanta GA,0.9716
1,INTERNATIONAL ASSOCIATION OF IT ASSET MANAGERS,IAITAM 4848 MUNSON STREET NW CANTON OH,CANTON,OH,44718,US,137673179,International Association of Information Techn...,4848 Munson St NW Canton OH,0.8279
6,"REINHART FOODSERVICE, L.L.C.-WISCONSIN",100 HARBOR VIEW PLAZA SUITE 200 LA CROSSE WI,LA CROSSE,WI,54601,US,47191968,"Reinhart Foodservice, L.L.C.",100 Harborview Plz La Crosse WI,0.7289
8,FWH,325 SUB WAY MILFORD CT,MILFORD,CT,6461,USA,34213823,"336 Bic Drive Realty II, LLC",325 Sub Way Milford CT,0.6667
4,CHRISTIAN D STORZ,FRANCHISE PERFORMANCE 9 CORONET TERRACE BURLIN...,BURLINGTON,NJ,8016,US,104955367,Vivian Storz,9 Coronet Ter Burlington NJ,0.6573
5,WIGGIN AND DANA LLP,ATTENTION: ACCOUNTS RECEIVABLE P.O. BOX 1832 N...,NEW HAVEN,CT,06508-1832,US,82672270,Wiggin & Dana LLP,265 Church St FL 14 New Haven CT,0.4829
7,MATTHEW MILLER,BUSINESS ANALYTICS 266 LEMUR DRIVE SAN ANTONIO TX,SAN ANTONIO,TX,78213,US,36768041,Methodist Plaza Imaging Ctr,San Antonio TX,0.3812
3,BRAINBOX DESIGN ESTRATEGICO S A,UnknownSupplierAddress,CURITBA,NaN,NaN,BRASIL,113036002,T & S Design LLC,1955 Greymont St Philadelphia PA,0.1396
0,NEXO,UnknownSupplierAddress,BRUSSELS,NaN,NaN,BELGUIM,927477331,"William Morris Endeavor Entertainment, LLC",1201 Demonbreun St FL 15 Nashville TN,0.0000


In [70]:
result.to_csv("Output D&B.csv")